In [2]:
import ctypes
import math
import random
from cached_property import cached_property

import os
import subprocess

In [47]:
def c_array(ctype, values):
    arr = (ctype*len(values))()
    arr[:] = values
    return arr

class BOX(ctypes.Structure):
    _fields_ = [("x", ctypes.c_float),
                ("y", ctypes.c_float),
                ("w", ctypes.c_float),
                ("h", ctypes.c_float)]
    
    def __repr__(self):
        return "BOX<{}, {}, {}, {}>".format(self.x, self.y, self.w, self.h)

class DETECTION(ctypes.Structure):
    _fields_ = [("bbox", BOX),
                ("classes", ctypes.c_int),
                ("prob", ctypes.POINTER(ctypes.c_float)),
                ("mask", ctypes.POINTER(ctypes.c_float)),
                ("objectness", ctypes.c_float),
                ("sort_class", ctypes.c_int)]


class IMAGE(ctypes.Structure):
    _fields_ = [("w", ctypes.c_int),
                ("h", ctypes.c_int),
                ("c", ctypes.c_int),
                ("data", ctypes.POINTER(ctypes.c_float))]
    
    def __repr__(self):
        return "IMAGE<{}, {}, {}>".format(self.w, self.h, self.c)
    
class NETWORK(ctypes.Structure):
    _fields_ = [
    ]
    
    def __repr__(self):
        return "NETWORK<>"

class METADATA(ctypes.Structure):
    _fields_ = [("classes", ctypes.c_int),
                ("names", ctypes.POINTER(ctypes.c_char_p))]
    
    
    def names_(self):
        pass
        
    
    def __repr__(self):
        return "METADATA<{}>".format(self.classes)

In [48]:
# darknet gets angry if you don't use the darknet directory as 'root'.
def chroot(f):
    def wrapper(self, *args, **kwargs):
        lwd = os.path.abspath(os.path.curdir)
        os.chdir(self.root)
        print(os.path.abspath(os.path.curdir))
        ret = f(self, *args, **kwargs)
        os.chdir(lwd)
        print(os.path.abspath(os.path.curdir))
        return ret
    return wrapper

class Darknet(object):
    def __init__(self, root=".."):
        self.root=os.path.abspath(root)
        
    @property
    def cfg_dir(self):
        return os.path.abspath(
            os.path.join(
                self.root,
                "cfg"
            )
        )
    
    @property
    def exe(self):
        return os.path.abspath(
            os.path.join(
                self.root,
                "darknet"
            )
        )
    
    @property
    def lib_path(self):
        return os.path.abspath(
            os.path.join(
                self.root,
                "libdarknet.so"
            )
        )
    
    @chroot
    def null(self):
        return ""
    
    @chroot
    def build(self, force=False, **kwargs):
        if os.path.exists(self.exe) and not force:
            return ""
        out = subprocess.check_output(["make", "clean"])
       
        for arg, value in kwargs.items():
            if value:     
                os.environ[arg.upper()]="1"
                #print("{}=1".format(arg.upper()))
            else:
                os.environ[arg.upper()]="0"
                #print("{}=0".format(arg.upper()))
                
        out = subprocess.check_output(["make", "-j8"])
        return (out.decode("UTF-8"))
    
    @cached_property
    def lib(self):
        lib_ = ctypes.CDLL(self.lib_path, ctypes.RTLD_GLOBAL)
        lib_.network_width.argtypes = [ctypes.c_void_p]
        lib_.network_width.restype = ctypes.c_int
        lib_.network_height.argtypes = [ctypes.c_void_p]
        lib_.network_height.restype = ctypes.c_int
        return lib_
    
    @chroot
    def load_meta(self, path):
        path = os.path.abspath(path)
       
        self.lib.get_metadata.argtypes = [ctypes.c_char_p]
        self.lib.get_metadata.restype = METADATA
        self.meta = self.lib.get_metadata(ctypes.c_char_p(path.encode()))
        
    @chroot
    def load_net(self, cfgfile, weightfile, clear=0):
        cfgfile=os.path.abspath(cfgfile)
        weightfile=os.path.abspath(weightfile)
                 
        self.lib.load_network.argtypes = [
            ctypes.c_char_p,
            ctypes.c_char_p,
            ctypes.c_int,
        ]
        self.lib.load_network.restype = ctypes.POINTER(NETWORK)
        
        
        cfgfile_=ctypes.c_char_p(cfgfile.encode("UTF-8"))
        weightfile_=ctypes.c_char_p(weightfile.encode("UTF-8"))
        clear_=ctypes.c_int(clear)
        self.net = self.lib.load_network(cfgfile_, weightfile_, clear_)
        
        
    def detect(self, image_path, thresh=.5, hier_thresh=.5, nms=.45):
        im = self.load_image(image_path)
        
        

        
        self.network_predict_image.argtypes = [ctypes.c_void_p, IMAGE]
        self.network_predict_image.restype = POINTER(ctypes.c_float)
        
        self.network_predict_image()
        
        
        """
        
        predict_image(net, im)
        
        num = ctypes.c_int(0)
        pnum = pointer(num)
        
        dets = get_network_boxes(net, im.w, im.h, thresh, hier_thresh, None, 0, pnum)
        num = pnum[0]
        if (nms): do_nms_obj(dets, num, meta.classes, nms);

        res = []
        for j in range(num):
            for i in range(meta.classes):
                if dets[j].prob[i] > 0:
                    b = dets[j].bbox
                    res.append((meta.names[i], dets[j].prob[i], (b.x, b.y, b.w, b.h)))
        res = sorted(res, key=lambda x: -x[1])
        free_image(im)
        free_detections(dets, num)
        return res"""
    
    
    def load_image(self, path, width=0, height=0, colors=0):
        path=os.path.abspath(path)
        load_image_ = self.lib.load_image_color
        load_image_.argtypes = [
            ctypes.c_char_p,
            ctypes.c_int,
            ctypes.c_int,
            ctypes.c_int,
        ]
        load_image_.restype = IMAGE
        
        path_ = ctypes.c_char_p(path.encode("UTF-8"))
        width_ = ctypes.c_int(width)
        height_ = ctypes.c_int(height)
        colors_ = ctypes.c_int(colors)
        
        img = load_image_(path_, width_, height_, colors_)
        return img
        
    def __repr__(self):
        return "Darknet<'{}'>".format(self.lib_path)

In [49]:
os.chdir("/tmp/")
dn = Darknet("/projects/darknet/")
out = dn.build(opencv=False, openmp=False)

/projects/darknet
/tmp


In [50]:
dn.load_meta("cfg/coco.data")

/projects/darknet
/tmp


In [51]:
dn.load_net("cfg/yolov3.cfg", "weights/yolov3.weights", 0)

/projects/darknet
/tmp


In [20]:
img = "/pictures/Vacation/Kids/IMG_0265.JPG"

In [21]:
img_ = dn.load_image(img)
img_

IMAGE<4000, 3000, 3>

In [ ]:
#lib = CDLL("/home/pjreddie/documents/darknet/libdarknet.so", RTLD_GLOBAL)

lib.network_width.argtypes = [c_void_p]
lib.network_width.restype = c_int
lib.network_height.argtypes = [c_void_p]
lib.network_height.restype = c_int

In [ ]:
predict = lib.network_predict
predict.argtypes = [c_void_p, POINTER(c_float)]
predict.restype = POINTER(c_float)

make_image = lib.make_image
make_image.argtypes = [c_int, c_int, c_int]
make_image.restype = IMAGE

get_network_boxes = lib.get_network_boxes
get_network_boxes.argtypes = [c_void_p, c_int, c_int, c_float, c_float, POINTER(c_int), c_int, POINTER(c_int)]
get_network_boxes.restype = POINTER(DETECTION)

make_network_boxes = lib.make_network_boxes
make_network_boxes.argtypes = [c_void_p]
make_network_boxes.restype = POINTER(DETECTION)

free_detections = lib.free_detections
free_detections.argtypes = [POINTER(DETECTION), c_int]

free_ptrs = lib.free_ptrs
free_ptrs.argtypes = [POINTER(c_void_p), c_int]

network_predict = lib.network_predict
network_predict.argtypes = [c_void_p, POINTER(c_float)]

reset_rnn = lib.reset_rnn
reset_rnn.argtypes = [c_void_p]

load_net = lib.load_network
load_net.argtypes = [c_char_p, c_char_p, c_int]
load_net.restype = c_void_p

do_nms_obj = lib.do_nms_obj
do_nms_obj.argtypes = [POINTER(DETECTION), c_int, c_int, c_float]

do_nms_sort = lib.do_nms_sort
do_nms_sort.argtypes = [POINTER(DETECTION), c_int, c_int, c_float]

free_image = lib.free_image
free_image.argtypes = [IMAGE]

letterbox_image = lib.letterbox_image
letterbox_image.argtypes = [IMAGE, c_int, c_int]
letterbox_image.restype = IMAGE

load_meta = lib.get_metadata
lib.get_metadata.argtypes = [c_char_p]
lib.get_metadata.restype = METADATA

load_image = lib.load_image_color
load_image.argtypes = [c_char_p, c_int, c_int]
load_image.restype = IMAGE

predict_image = lib.network_predict_image
predict_image.argtypes = [c_void_p, IMAGE]
predict_image.restype = POINTER(c_float)

def classify(net, meta, im):
    out = predict_image(net, im)
    res = []
    for i in range(meta.classes):
        res.append((meta.names[i], out[i]))
    res = sorted(res, key=lambda x: -x[1])
    return res

def detect(net, meta, image, thresh=.5, hier_thresh=.5, nms=.45):
    im = load_image(image, 0, 0)
    num = c_int(0)
    pnum = pointer(num)
    predict_image(net, im)
    dets = get_network_boxes(net, im.w, im.h, thresh, hier_thresh, None, 0, pnum)
    num = pnum[0]
    if (nms): do_nms_obj(dets, num, meta.classes, nms);

    res = []
    for j in range(num):
        for i in range(meta.classes):
            if dets[j].prob[i] > 0:
                b = dets[j].bbox
                res.append((meta.names[i], dets[j].prob[i], (b.x, b.y, b.w, b.h)))
    res = sorted(res, key=lambda x: -x[1])
    free_image(im)
    free_detections(dets, num)
    return res

In [ ]:
import os

In [ ]:
net = load_net(os.path.abspath("../cfg/yolov3.cfg").encode(), os.path.abspath("../cfg/yolov3.weights").encode(), 0)

In [ ]:
dog_jpg = os.path.abspath("../data/dog.jpg")
assert os.path.exists(dog_jpg)

In [ ]:
dog = load_image(dog_jpg.encode(), 0, 0)

In [ ]:
num = c_int(0)
pnum = pointer(num)
predict_image(net, dog)

In [ ]:
meta = load_meta(os.path.abspath("../cfg/coco.data").encode())

In [ ]:
if __name__ == "__main__":
    #net = load_net("cfg/densenet201.cfg", "/home/pjreddie/trained/densenet201.weights", 0)
    #im = load_image("data/wolf.jpg", 0, 0)
    #meta = load_meta("cfg/imagenet1k.data")
    #r = classify(net, meta, im)
    #print r[:10]
    net = load_net("cfg/tiny-yolo.cfg", "tiny-yolo.weights", 0)
    meta = load_meta("cfg/coco.data")
    r = detect(net, meta, "data/dog.jpg")
    print r
    

In [ ]:
def classify(net, meta, im):
    out = predict_image(net, im)
    res = []
    for i in range(meta.classes):
        res.append((meta.names[i], out[i]))
    res = sorted(res, key=lambda x: -x[1])
    return res
